# Introdução

Na aula passa abordamos um processo de busca automática pelos melhores hiperparâmetros.Na prática, para usar esse método precisamos:

1. Dividir os dados em treino e teste. Lembrando que, para conjuntos maiores (>1MM) dividimos em treino, teste e validação. 
2. Definir uma malha de hiperparâmetros (ver sempre na biblioteca os hiperparâmetros dos algoritmos que se deseja utilizar)
3. Definir o tipo de validaçao cruzada (se for classificação, usar StratifiedKFold)
4. Definir um método de busca (até agora vimos que Grid Search e hoje veremos Randomized Search)

Para cada combinação de hiperparâmetros definidos pelas malhas, o algoritmo realiza uma validação cruzada, o que lhe permitirá estimar o erro em produção para aquele conjunto específico. No final, ele nos retorna o conjunto de hiperparâmetros para o qual o erro estimado é mínimo.

No final, devemos sempre treinar nossos dados no X_train e avaliar a performance no X_test, ainda não visto. Essa será a performance estimada em produção.

OBS 1 : Caso o conjunto de dados seja relativamente grande, não será realizada validação cruzada. Apenas um processo iterativo e automático de busca será realizado. Nesse caso, o algoritmo retorna pra gente a combinação que produz o menor erro no conjunto de validação. No final, treinamos o modelo no X_train e estimamos a performance no final utilizando o X_test.

OBS 2: Em um conjunto menor de dados, seria interessante utilizar um método de validação conhecido como Nested Cross Validation. Para mais informações, acessem o [link](https://weina.me/nested-cross-validation/)

OBS 3: No caso de dados grandes, não precisamos da validaçao cruzada, então não faz sentido utilizar GridSearchCV ou RandomizedSearchCV. Podemos, em vez disso, utilizar as classes ParameterGrid e ParameterSampler do sklearn para realizar nosso processo de busca.

# ParameterGrid

Definimos a grid de parâmetros da mesma forma que aprendemos, mas não usaremos GridSearchCV.

Em vez disso, utilizaremos a clase ParameterGrid.

In [16]:
from sklearn.model_selection import ParameterGrid
from sklearn.neighbors import KNeighborsClassifier

In [17]:
# grid de hiperparâmetros
param_grid = {'n_neighbors': [1, 2]}

In [18]:
param_grid

{'n_neighbors': [1, 2]}

A partir daí, criamos o grid com todas as combinações

In [19]:
ParameterGrid(param_grid)

In [22]:
validation_error = []
for combinacao in ParameterGrid(param_grid):
    knn = KNeighborsClassifier(**combinacao)
    knn.fit(X_train, y_train)
    y_val = knn.predict(X_test)
    erro = accuracy_score(y_test, y_val)
    validation_error.append(erro)
    print(combinacao)

{'n_neighbors': 1}
{'n_neighbors': 2}


Na prática, não faremos validação cruzada. Apenas iremos treinar e validar (**validar significa avaliar o erro no dataset de validação!**) um modelo para cada combinação do dicionário acima e checar qual retorna para nós o menor erro de validação. Com isso, treinamos nosso modelo no```X_train```imamos a performance dele no ```X_Test```.

# Randomized Search CV

O Grid Search pode ser considerado um método por exaustão para escolher o melhor conjunto de hiperparâmetros. No Grid Search, o cientista de dados configura uma grade de valores de hiperparâmetros e, para cada combinação, treina um modelo e pontua nos dados de teste. Nessa abordagem, todas as combinações de valores de hiperparâmetros são testadas, o que pode ser muito ineficiente. Por exemplo, pesquisar 20 valores de parâmetros diferentes para cada um dos 4 parâmetros exigirá 160.000 tentativas de validação cruzada. Isso equivale a 1.600.000 ajustes de modelo e 1.600.000 previsões se a validação cruzada de 10 vezes for usada. Embora o Scikit Learn ofereça a função GridSearchCV para simplificar o processo, seria uma execução extremamente cara em termos de capacidade e tempo de computação.
Em contraste,o Randomized Search configura uma grade de valores de hiperparâmetros e seleciona combinações aleatórias para treinar o modelo e avaliar o erro. Isso permite que você controle explicitamente o número de combinações de parâmetros que são tentadas. O número de iterações de pesquisa é definido com base no tempo ou recursos. O Scikit Learn oferece a função RandomizedSearchCV para esse processo.
Embora seja possível que RandomizedSearchCV não encontre um resultado tão preciso quanto GridSearchCV, ele surpreendentemente escolhe o melhor resultado com mais frequência do que não e em uma fração do tempo que GridSearchCV levaria. Com os mesmos recursos, o Random Search pode até superar o Grid Search. Isso pode ser visualizado no gráfico abaixo quando parâmetros contínuos são usados.

<img src="https://miro.medium.com/max/1400/1*9W1MrRkHi0YFmBoHi9Y2Ow.png" width=800>

O RandomizedSearch é baseado em distribuições, ou seja, ele pode selecionar valores entre um intervalo de valores, diferentemente do GridSearch.

## ParameterSampler

Quando queremos usar uma grid aleatória (Random Search), mas sem validação cruzada, podemos usar a classe ```ParameterSampler``` do sklearn. Essa classe vai receber o dicionário com as distribuições dos parâmetros e o número de iterações. 
A cada iteração em um loop, ele vai gerar um valor aleatório de combinação de hiperparâmetros.

In [23]:
from sklearn.model_selection import ParameterSampler
from scipy.stats import loguniform, uniform, randint


In [24]:
param_grid = dict()
param_grid['solver'] = ['newton-cg', 'lbfgs', 'liblinear']
param_grid['penalty'] = ['none', 'l1', 'l2', 'elasticnet']
param_grid['C'] = loguniform(1e-5, 100)
param_grid['whatever'] = randint(1, 1000)

In [25]:
ParameterSampler(param_grid, n_iter=10, random_state=123)

In [26]:
for combinacao in ParameterSampler(param_grid, n_iter=10, random_state=123):
    
    print(combinacao)

{'C': 0.750385268090156, 'penalty': 'l2', 'solver': 'liblinear', 'whatever': 989}
{'C': 0.6857944800896927, 'penalty': 'l1', 'solver': 'liblinear', 'whatever': 124}
{'C': 2.8853223070403433, 'penalty': 'l1', 'solver': 'newton-cg', 'whatever': 114}
{'C': 0.023255373037796706, 'penalty': 'l1', 'solver': 'newton-cg', 'whatever': 943}
{'C': 0.24616089675634506, 'penalty': 'l1', 'solver': 'liblinear', 'whatever': 254}
{'C': 0.7299384488053402, 'penalty': 'none', 'solver': 'newton-cg', 'whatever': 818}
{'C': 0.00018942710113933008, 'penalty': 'l2', 'solver': 'newton-cg', 'whatever': 40}
{'C': 0.9689720939864422, 'penalty': 'elasticnet', 'solver': 'newton-cg', 'whatever': 958}
{'C': 8.831257714012057, 'penalty': 'l1', 'solver': 'newton-cg', 'whatever': 861}
{'C': 1.140522028920257, 'penalty': 'l1', 'solver': 'lbfgs', 'whatever': 631}


# Prática Guiada

In [1]:
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV, cross_val_score, train_test_split, StratifiedKFold
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
%matplotlib inline

In [2]:
# separando x e y
df = load_iris()
X = df.data
y = df.target
# Split de treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state=123)
len(X_train), len(X_test), len(y_train), len(y_test)

(120, 30, 120, 30)

In [4]:
# stratified kfold
skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=123)

In [5]:
from sklearn.model_selection import RandomizedSearchCV

In [6]:
param_grid = dict(n_neighbors=range(1,31))
knn = KNeighborsClassifier()


In [7]:
param_grid

{'n_neighbors': range(1, 31)}

In [8]:
# random search
random_search = RandomizedSearchCV(knn, param_grid, n_iter=10, cv=skf, scoring='neg_log_loss', verbose=1, random_state=123)

In [9]:
random_search.fit(X_train, y_train)

Fitting 10 folds for each of 10 candidates, totalling 100 fits


RandomizedSearchCV(cv=StratifiedKFold(n_splits=10, random_state=123, shuffle=True),
                   estimator=KNeighborsClassifier(),
                   param_distributions={'n_neighbors': range(1, 31)},
                   random_state=123, scoring='neg_log_loss', verbose=1)

In [10]:
random_search.best_params_

{'n_neighbors': 12}

In [11]:
random_search.best_score_

-0.07803933303339768

In [12]:
# grid search
grid_search = GridSearchCV(knn, param_grid, cv=skf, scoring='neg_log_loss', verbose=1)

In [13]:
grid_search.fit(X_train, y_train)

Fitting 10 folds for each of 30 candidates, totalling 300 fits


GridSearchCV(cv=StratifiedKFold(n_splits=10, random_state=123, shuffle=True),
             estimator=KNeighborsClassifier(),
             param_grid={'n_neighbors': range(1, 31)}, scoring='neg_log_loss',
             verbose=1)

In [14]:
grid_search.best_params_

{'n_neighbors': 12}

In [15]:
grid_search.best_score_

-0.07803933303339768

Note que o grid search demorou mais que o dobro de tempo para rodar. No final, o random search conseguiu encontrar o melhor número de vizinhos mais próximos, com menos modelos.